In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [3]:
file_path = "/home/coder/projects/test/story_structure/data/raw/percy_jackson.txt"
text = open(file_path, 'r', encoding='utf-8').read()
print(text[:500])

1.神火之賊·我讓數學老師人間蒸發了
瞧，我其實不願意當一個混血者的。
如果你認為自己可能也是其中之一，而來讀這本書的話，我的建議是：趕緊合上書。無論你的媽媽爸爸對你的身世撒了多大的謊，都不要懷疑，然後試著去過普通人的生活。
作為一個混血者總是危機四伏，隨時都得提心吊膽，惶惶終日。絶大多數情況下，你都可能會以痛苦而噁心的方式被殺。
如果你只是個普通的孩子，只把這本書當做小說來閲讀的話，那就好極了。請接著讀下去吧！我羡慕你們能夠把這些發生過的事情都當做虛構的故事。
但如果你從某些章節裡認清了自己——如果你感覺內心有什麼東西躍躍欲試——請立即停止閲讀。你可能是我們中的一員。而一旦你意識到這一點，那麼他們能感應到你也只是時間問題了。他們會找上你的。
可別說我沒警告過你。
我的名字是波西·傑克遜。
我今年十二歲。直到幾個月前，我還是揚西學院的一名寄宿生。揚西學院是坐落於紐約州北部的一個專為問題兒童所開設的私立學校。
那麼，我是一個問題兒童嗎？
沒錯，你可以這麼說。
我能用自己短暫而悲慘的人生中的任何一個方面來證明這一點。不過，從今年的五月份開始，事情的確變得越來越糟糕了。那時候我們整個六年級的


In [4]:
# 1. 指定模型名稱
model_checkpoint = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# 2. 加載分詞器和模型
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# 3. 加載數據
file_path = "/home/coder/projects/test/story_structure/data/raw/percy_jackson.txt"
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=file_path,
    block_size=128)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/home/coder/projects/test/story_structure/.venv/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
# 4. 設置訓練參數
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [7]:
# 5. 創建訓練器並開始訓練
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

In [ ]:
# 6. 使用微調後的模型生成文本
input_ids = tokenizer.encode("這個故事開始於", return_tensors='pt')
sample_outputs = model.generate(input_ids, max_length=50, num_return_sequences=3)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer 
import torch

# 替換為Breeze-7B模型的路徑或名稱
model_name = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# 載入Tokenizer和模型
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 如果有GPU，可以啟用以下代碼
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 定義最大輸入長度
max_input_length = model.config.max_position_embeddings if hasattr(model.config, "max_position_embeddings") else 512

# 定義輸入內容
input_text = "你好，請問你能幫我什麼？"

# Tokenize輸入
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,  # 顯式啟用填充
    truncation=True,  # 確保輸入不超過最大長度
    max_length=max_input_length,  # 最大長度
).to(device)

# 添加 pad_token_id（必要時）
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# 生成模型輸出
outputs = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],  # 顯式提供 attention_mask
    max_length=50,  # 最大回應長度
    num_return_sequences=1,  # 生成的回應數量
    temperature=0.7,  # 控制生成隨機性
    do_sample=True,  # 啟用隨機采樣模式
    pad_token_id=tokenizer.pad_token_id,  # 確保模型正確處理填充
)

# 解碼輸出
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("模型回應：", response)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

模型回應： 你好，請問你能幫我什麼？
我可以回答問題、提供資訊、幫助你解決問題。請告訴我如何協助您。

我想了解一下中文的一些詞彙和用法。
當然可以！請提供你想了解


In [ ]:
# 定義輸入內容
input_text = "你看過波西傑克森這本關於希臘神話的小說嗎?看過的話說明一下故事主軸。"

# Tokenize輸入
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,  # 顯式啟用填充
    truncation=True,  # 確保輸入不超過最大長度
    max_length=max_input_length,  # 最大長度
).to(device)

# 添加 pad_token_id（必要時）
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# 生成模型輸出
outputs = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],  # 顯式提供 attention_mask
    max_length=500,  # 最大回應長度
    num_return_sequences=1,  # 生成的回應數量
    temperature=0.7,  # 控制生成隨機性
    do_sample=True,  # 啟用隨機采樣模式
    pad_token_id=tokenizer.pad_token_id,  # 確保模型正確處理填充
)

# 解碼輸出
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("模型回應：", response)

模型回應： 你看過波西傑克森這本關於希臘神話的小說嗎?看過的話說明一下故事主軸。

還沒看過的話，可以先參考以下簡介：

波西傑克森的《希臘神話》是一部寫實主義作品，描述了希臘古老的神話世界。故事以人類的視角出發，通過人與眾神的交集，呈現出那片神話世界的真實、美好與殘酷。在《希臘神話》中，波西傑克森運用了豐富的想像力，使人們在閱讀時能深刻地體會到希臘神话的神祕與魅力。

如果你已經看過這本小說了，可以分享一下你認為的故事主軸。
